In [6]:
import pygame, sys, random
import numpy as np
import math
import matplotlib.pyplot as plt
import pylab
from RVO_Py_MAS.RVO import RVO_update, reach, compute_V_des
from RVO_Py_MAS.vis import visualize_traj_dynamic

num_episodes = 250

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 100]
initPosAgent = initPosAgentStandard
goalPos = [100, 900]
rangeNumber = 5
obsNumber = 200

In [7]:
ws_model = dict()
ws_model['robot_radius'] = agentRadius
ws_model['circular_obstacles'] = []
ws_model['boundary'] = []

In [8]:
def rangeFinder(allObsPos, rangeCenter):
    indexMin = []
    tmpDist = [0 for _ in range(obsNumber)]
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
        tmpDist[i] = allObsAgtDistance[i]
    tmpDist.sort()
    for i in range(0, rangeNumber):
        indexMin.append(allObsAgtDistance.index(tmpDist[i]))
    return indexMin

In [9]:
#### pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([220, 220, 220])

rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
#for i in range(0,obsNumber):
#    obstaclePos[i][0] = int(150 + i % 10 * 100)
#    obstaclePos[i][1] = int(i / 10 * 100)
# Number of Obstacles = 150
# Number of Obstacles = 200
for i in range(0,obsNumber):
    obstaclePos[i][0] = int(150 + i % 14 * 80)
    obstaclePos[i][1] = int(i / 14 * 80)

index = rangeFinder(obstaclePos, initPosAgent)
rangeObstacle = [[0,0] for _ in range(rangeNumber)]
for j in range(0, rangeNumber):
    rangeObstacle[j][0] = obstaclePos[index[j]][0]
    rangeObstacle[j][1] = obstaclePos[index[j]][1]

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    
    X = [initPosAgent] + rangeObstacle
    V = [[0,0] for i in xrange(len(X))]
    V_max = [5.0 for i in xrange(len(X))]
#     goal = [goalPos] + rangeObstacle
    randomNo = [0 for _ in range(rangeNumber)]
    goal = [goalPos]
    for i in range(0, rangeNumber):
        randomNo[i] = random.randrange(0, 100)
        goal += [initPosAgent]

    print("Episode ", e, "Starts!")
    while not done:
        V_des = compute_V_des(X, goal, V_max)
        V = RVO_update(X, V_des, V, ws_model)

        x += int(V[0][0])
        y += int(V[0][1])
        
        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
        
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius * 2:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        index = rangeFinder(obstaclePos, [x,y])
        for j in range(0, rangeNumber):
#             V[j + 1][0] = random.randrange(-1, 2)
#             V[j + 1][1] = random.randrange(-1, 2)
            rangeObstacle[j][0] = obstaclePos[index[j]][0] + int(round(V[j + 1][0]))
            rangeObstacle[j][1] = obstaclePos[index[j]][1] + int(round(V[j + 1][1]))
            obstaclePos[index[j]][0] = rangeObstacle[j][0]
            obstaclePos[index[j]][1] = rangeObstacle[j][1]
            
            if math.sqrt((x - rangeObstacle[j][0])**2 + (y - rangeObstacle[j][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                done = True
        k = 0
        for i in range(0,obsNumber):

            
            if i == index[0] or i == index[1] or i == index[2] or i == index[3] or i == index[4]:
                pygame.draw.circle(screen, [255, 255, 50], rangeObstacle[k], obstacleRadius, 0)
                k += 1
            else:
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
        goal = [goalPos]
        for i in range(0, rangeNumber):
            goal += [[rangeObstacle[i][0] + random.randrange(-1,2), rangeObstacle[i][1] + random.randrange(-1,2)]]
#             if i == rangeNumber - 1:
#                 goal += [rangeObstacle[0]]
#             else:
#                 goal += [rangeObstacle[i + 1]]
        X = [[x, y]] + rangeObstacle
#         goal = [goalPos, obstaclePos[index[0]], obstaclePos[index[1]], obstaclePos[index[2]], obstaclePos[index[3]], obstaclePos[index[4]]]

        if done:
            if collisionFlag == 1:
                rList.append(1)
            elif collisionFlag == -1:
                rList.append(0)
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
#            for i in range(0,obsNumber):
#                obstaclePos[i][0] = int(150 + i % 10 * 100)
#                obstaclePos[i][1] = int(i / 10 * 100)
            # Number of Obstacles = 150
            # Number of Obstacles = 200
            for i in range(0,obsNumber):
                obstaclePos[i][0] = int(150 + i % 14 * 80)
                obstaclePos[i][1] = int(i / 14 * 80)

            episodes.append(e)
            
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 0)
        pygame.display.flip()
        screen.fill([220,220,220])
    print("Percent of successful episodes: " + str(100.0 * sum(rList)/(e + 1)) + "%")

('Episode ', 0, 'Starts!')
Goal Reached!
Percent of successful episodes: 100.0%
('Episode ', 1, 'Starts!')
Collision!
Percent of successful episodes: 50.0%
('Episode ', 2, 'Starts!')
Goal Reached!
Percent of successful episodes: 66.6666666667%
('Episode ', 3, 'Starts!')
Goal Reached!
Percent of successful episodes: 75.0%
('Episode ', 4, 'Starts!')
Goal Reached!
Percent of successful episodes: 80.0%
('Episode ', 5, 'Starts!')
Goal Reached!
Percent of successful episodes: 83.3333333333%
('Episode ', 6, 'Starts!')
Goal Reached!
Percent of successful episodes: 85.7142857143%
('Episode ', 7, 'Starts!')
Goal Reached!
Percent of successful episodes: 87.5%
('Episode ', 8, 'Starts!')
Goal Reached!
Percent of successful episodes: 88.8888888889%
('Episode ', 9, 'Starts!')
Goal Reached!
Percent of successful episodes: 90.0%
('Episode ', 10, 'Starts!')
Goal Reached!
Percent of successful episodes: 90.9090909091%
('Episode ', 11, 'Starts!')
Goal Reached!
Percent of successful episodes: 91.6666666667

Collision!
Percent of successful episodes: 72.9166666667%
('Episode ', 96, 'Starts!')
Goal Reached!
Percent of successful episodes: 73.1958762887%
('Episode ', 97, 'Starts!')
Collision!
Percent of successful episodes: 72.4489795918%
('Episode ', 98, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.7272727273%
('Episode ', 99, 'Starts!')
Goal Reached!
Percent of successful episodes: 73.0%
('Episode ', 100, 'Starts!')
Collision!
Percent of successful episodes: 72.2772277228%
('Episode ', 101, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.5490196078%
('Episode ', 102, 'Starts!')
Collision!
Percent of successful episodes: 71.8446601942%
('Episode ', 103, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.1153846154%
('Episode ', 104, 'Starts!')
Collision!
Percent of successful episodes: 71.4285714286%
('Episode ', 105, 'Starts!')
Goal Reached!
Percent of successful episodes: 71.6981132075%
('Episode ', 106, 'Starts!')
Collision!
Percent of successful episo

Goal Reached!
Percent of successful episodes: 72.4867724868%
('Episode ', 189, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.6315789474%
('Episode ', 190, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.7748691099%
('Episode ', 191, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.9166666667%
('Episode ', 192, 'Starts!')
Goal Reached!
Percent of successful episodes: 73.0569948187%
('Episode ', 193, 'Starts!')
Collision!
Percent of successful episodes: 72.6804123711%
('Episode ', 194, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.8205128205%
('Episode ', 195, 'Starts!')
Collision!
Percent of successful episodes: 72.4489795918%
('Episode ', 196, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.5888324873%
('Episode ', 197, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.7272727273%
('Episode ', 198, 'Starts!')
Goal Reached!
Percent of successful episodes: 72.864321608%
('Episode ', 199, 'Starts!')
Collision!
Percen

In [2]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

NameError: name 'rList' is not defined